In [1]:
import os, csv, pandas as pd
from py2neo import Graph

In [2]:
files_path = "../Output/DataLLM"
files = os.listdir(files_path)

In [3]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "@Martiale01"))

In [4]:
for data in files :
    df = pd.read_csv(f"{files_path}/{data}", delimiter=",")
    for _, row in df.iterrows():
        odd = row['ODD']
        cible = row['Cible']
        indicateur = row['Indicateur']
        mot_cle = row['Mots-cles']

         # Nous utilisons MERGE au lieu de CREATE afin de créer les nœuds ODD, Cible, Indicateur, Mots-clés sans doublons. Si le nœud existe déjà, il ne sera pas recréé
        graph.run(f"MERGE (o:ODD {{code: '{odd[:5]}'}}) SET o.titre = '{odd[8:]}'")
        graph.run(f"MERGE (c:Cible {{code: '{cible[6:12]}'}}) SET c.titre = '{cible[14:]}'")
        graph.run(f"MERGE (i:Indicateur {{code: '{indicateur[11:19]}'}}) SET i.titre = '{indicateur[22:]}'")
        graph.run(f'MERGE (m:MotCle {{mot: "{mot_cle}"}})')

        # Les relations entre les nœuds
        graph.run(f"""
            MATCH (o:ODD {{code: '{odd[:5]}'}}),
                  (c:Cible {{code: '{cible[6:12]}'}}),
                  (i:Indicateur {{code: '{indicateur[11:19]}'}}),
                  (m:MotCle {{mot: '{mot_cle}'}})
            MERGE (o)-[:CONTIENT]->(c)
            MERGE (c)-[:CONTIENT]->(i)
            MERGE (i)-[:CONTIENT]->(m)
        """)


In [5]:
Ontology_final = "../Other/Ontology_final.csv"

with open(Ontology_final, encoding="utf-8") as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter=';')
    data = list(csvreader)

df_Ontology_final = pd.DataFrame(data)

In [6]:
for _, row in df_Ontology_final.iterrows():
    mot_cle = row['﻿keyword']
    classification = row['classification']
    graph.run(f'MERGE (k:Keyword {{mot: "{mot_cle}"}})')
    graph.run(f"MERGE (g:Goal {{titre: '{classification}'}})")
    graph.run(f"""
    MATCH (k:Keyword {{mot: "{mot_cle}"}}), (g:Goal {{titre: "{classification}"}})
    MERGE (g)-[:CONTIENT]->(k)
    """)
